In [3]:
############################ Harris filter
import cv2
import time
import sys
import numpy as np
sys.path.append("../modules/")
import input_output
import processor

def track_nothing(val):
    pass

def on_gamma_trackbar(val):
    det.processors ["cam"] ["gamma correction"].set_gamma (float (val) / 100)
    
def applyHarris(img):
    blockSize = cv2.getTrackbarPos ("blocksize", "trackbars")
    ksize = cv2.getTrackbarPos ("ksize", "trackbars")
    if ksize % 2 == 0:
        ksize = ksize + 1
    k = cv2.getTrackbarPos ("k", "trackbars") / 100
    thrshld = cv2.getTrackbarPos ("thrshld", "trackbars") / 100000
    dst = cv2.cornerHarris (img, blockSize, ksize, k)
    img [np.where (dst > thrshld)] = 255
    return img    

det = processor.Processors ()
det.add_processor ("cam")

det.add_filter (processor.gamma_correction (1), "cam", "gamma correction")
det.add_filter (processor.colorspace_to_colorspace ("RGB", "GRAY"), "cam", "colorspaces change")
det.add_filter (processor.custom_operation (applyHarris, "harris filter"), "cam", "harris filter")

cv2.namedWindow ('trackbars', cv2.WINDOW_NORMAL | cv2.WINDOW_AUTOSIZE)
cv2.createTrackbar ("gamma", "trackbars", 100, 200, on_gamma_trackbar)

cv2.createTrackbar ("blocksize", "trackbars", 3, 10, track_nothing)
cv2.createTrackbar ("ksize", "trackbars", 5, 10, track_nothing)
cv2.createTrackbar ("k", "trackbars", 4, 20, track_nothing)
cv2.createTrackbar ("thrshld", "trackbars", 10, 100, track_nothing)

print ('Start init of the front camera')
start = time.time()
try:
    source  = input_output.Source ("1")
except:
    print ("Cam not found")
    source.release()    
end = time.time()
print ('End of front camera init. Duration {sec} seconds.'.format(sec = end - start))
#frame = source.get_frame ()
#image, _ = det.process (frame, "cam")

while (True):     
    frame = source.get_frame ()
    image, _ = det.process (frame, "cam")
    cv2.imshow ('front cam', image)
    key = cv2.waitKey (1)
    if key == 27:  # wait for ESC key to exit
         break
source.release()
cv2.destroyAllWindows()

Start init of the front camera
End of front camera init. Duration 3.5609474182128906 seconds.


In [4]:
################################# BBOX RGB
import cv2
import time
import os
import math
import sys
import numpy as np

sys.path.append("../modules/")

import input_output
import processor

def track_nothing(val):
    pass

det = processor.Processors ()
det.add_processor ("cam")

def on_gamma_trackbar(val):
    det.processors ["cam"] ["gamma correction"].set_gamma (float (val) / 100)

det.add_filter (processor.gamma_correction (1), "cam", "gamma correction") 
det.add_filter (processor.inrange ((0,0,0), (255,255,255)), "cam", "inrange")
det.add_filter (processor.morphology ("open", 1), "cam", "morph_open")
det.add_filter (processor.morphology ("close", 1), "cam", "morph_close")
det.add_filter (processor.morphology ("erode", 1), "cam", "morph_erode")
det.add_filter (processor.morphology ("dilate", 1), "cam", "morph_dilate")
det.add_filter (processor.filter_connected_components (), "cam", "bbox")
det.add_filter (processor.max_area_cc_bbox (), "cam", "bbox")

cv2.namedWindow ('trackbars', cv2.WINDOW_NORMAL)
cv2.createTrackbar ("gamma", "trackbars", 100, 200, on_gamma_trackbar)

cv2.createTrackbar ("thr_l_r", "trackbars", 100, 255, track_nothing)
cv2.createTrackbar ("thr_l_g", "trackbars", 100, 255, track_nothing)
cv2.createTrackbar ("thr_l_b", "trackbars", 100, 255, track_nothing)
cv2.createTrackbar ("thr_h_r", "trackbars", 100, 255, track_nothing)
cv2.createTrackbar ("thr_h_g", "trackbars", 100, 255, track_nothing)
cv2.createTrackbar ("thr_h_b", "trackbars", 100, 255, track_nothing)

cv2.createTrackbar ("morph_op", "trackbars", 1, 10, track_nothing)
cv2.createTrackbar ("morph_cl", "trackbars", 1, 10, track_nothing)
cv2.createTrackbar ("morph_er", "trackbars", 1, 10, track_nothing)
cv2.createTrackbar ("morph_di", "trackbars", 1, 10, track_nothing)

print ('Start init of the front camera')
start = time.time()
try:
    source  = input_output.Source ("1")
except:
    print ("Cam not found")
    source.release()    
end = time.time()
print ('End of front camera init. Duration {sec} seconds.'.format(sec = end - start))

while (True):     
    det.processors ["cam"] ["inrange"].set_ths (
        (cv2.getTrackbarPos ("thr_l_r", "trackbars"),
         cv2.getTrackbarPos ("thr_l_g", "trackbars"),
         cv2.getTrackbarPos ("thr_l_b", "trackbars")),
        (cv2.getTrackbarPos ("thr_h_r", "trackbars"),
         cv2.getTrackbarPos ("thr_h_g", "trackbars"),
         cv2.getTrackbarPos ("thr_h_b", "trackbars")))
    det.processors ["cam"] ["morph_open"].ker_sz = cv2.getTrackbarPos ("morph_op", "trackbars")
    det.processors ["cam"] ["morph_close"].ker_sz = cv2.getTrackbarPos ("morph_cl", "trackbars")
    det.processors ["cam"] ["morph_erode"].ker_sz = cv2.getTrackbarPos ("morph_er", "trackbars")
    det.processors ["cam"] ["morph_dilate"].ker_sz = cv2.getTrackbarPos ("morph_di", "trackbars")
    
    frame = source.get_frame ()
    _,_ = det.process (frame, "cam")
    
    stages_picts = det.get_stages_picts ("cam")
    cv2.imshow ("cam", input_output.form_grid(stages_picts, window_x_sz=1000))
    
    key = cv2.waitKey (1)
    if key == 27:  # wait for ESC key to exit
         break
source.release()
cv2.destroyAllWindows()

Start init of the front camera
End of front camera init. Duration 11.664645433425903 seconds.


In [5]:
################################# BBOX HSV
import cv2
import time
import os
import math
import sys
import numpy as np

sys.path.append("../modules/")

import input_output
import processor

def track_nothing(val):
    pass

det = processor.Processors ()
det.add_processor ("cam")

def on_gamma_trackbar(val):
    det.processors ["cam"] ["gamma correction"].set_gamma (float (val) / 100)

det.add_filter (processor.gamma_correction (1), "cam", "gamma correction") 
det.add_filter (processor.colorspace_to_colorspace("RGB", "HSV"), "cam", "rgb2hsv")
det.add_filter (processor.inrange ((0,0,0), (255,255,255)), "cam", "inrange")
det.add_filter (processor.morphology ("open", 1), "cam", "morph_open")
det.add_filter (processor.morphology ("close", 1), "cam", "morph_close")
det.add_filter (processor.morphology ("erode", 1), "cam", "morph_erode")
det.add_filter (processor.morphology ("dilate", 1), "cam", "morph_dilate")
det.add_filter (processor.filter_connected_components (), "cam", "bbox")
det.add_filter (processor.max_area_cc_bbox (), "cam", "bbox")

cv2.namedWindow ('trackbars', cv2.WINDOW_NORMAL)
cv2.createTrackbar ("gamma", "trackbars", 100, 200, on_gamma_trackbar)

cv2.createTrackbar ("thr_l_h", "trackbars", 100, 255, track_nothing)
cv2.createTrackbar ("thr_l_s", "trackbars", 100, 255, track_nothing)
cv2.createTrackbar ("thr_l_v", "trackbars", 100, 255, track_nothing)
cv2.createTrackbar ("thr_h_h", "trackbars", 100, 255, track_nothing)
cv2.createTrackbar ("thr_h_s", "trackbars", 100, 255, track_nothing)
cv2.createTrackbar ("thr_h_v", "trackbars", 100, 255, track_nothing)

cv2.createTrackbar ("morph_op", "trackbars", 1, 10, track_nothing)
cv2.createTrackbar ("morph_cl", "trackbars", 1, 10, track_nothing)
cv2.createTrackbar ("morph_er", "trackbars", 1, 10, track_nothing)
cv2.createTrackbar ("morph_di", "trackbars", 1, 10, track_nothing)

print ('Start init of the front camera')
start = time.time()
try:
    source  = input_output.Source ("1")
except:
    print ("Cam not found")
    source.release()    
end = time.time()
print ('End of front camera init. Duration {sec} seconds.'.format(sec = end - start))

while (True):     
    det.processors ["cam"] ["inrange"].set_ths (
        (cv2.getTrackbarPos ("thr_l_h", "trackbars"),
         cv2.getTrackbarPos ("thr_l_s", "trackbars"),
         cv2.getTrackbarPos ("thr_l_v", "trackbars")),
        (cv2.getTrackbarPos ("thr_h_h", "trackbars"),
         cv2.getTrackbarPos ("thr_h_s", "trackbars"),
         cv2.getTrackbarPos ("thr_h_v", "trackbars")))
    det.processors ["cam"] ["morph_open"].ker_sz = cv2.getTrackbarPos ("morph_op", "trackbars")
    det.processors ["cam"] ["morph_close"].ker_sz = cv2.getTrackbarPos ("morph_cl", "trackbars")
    det.processors ["cam"] ["morph_erode"].ker_sz = cv2.getTrackbarPos ("morph_er", "trackbars")
    det.processors ["cam"] ["morph_dilate"].ker_sz = cv2.getTrackbarPos ("morph_di", "trackbars")
    
    frame = source.get_frame ()
    _,_ = det.process (frame, "cam")
    
    stages_picts = det.get_stages_picts ("cam")
    cv2.imshow ("cam", input_output.form_grid(stages_picts, window_x_sz=1000))
    
    key = cv2.waitKey (1)
    if key == 27:  # wait for ESC key to exit
         break
source.release()
cv2.destroyAllWindows()

Start init of the front camera
End of front camera init. Duration 11.941836833953857 seconds.
